# setup-demo

デモセットアップ用のノートブックです。  
ベクトルデータベース（Oracle Database 23ai）のセットアップやそれを活用した RAG を試験的に試すことができます。

ロガーの設定を行います。

In [ ]:
import logging
import os
import sys
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

logger = logging.getLogger(__name__)
handler = logging.StreamHandler(sys.stdout)
log_level = os.getenv("LOG_LEVEL", "ERROR").upper()
handler.setLevel(log_level)
logger.setLevel(log_level)

デモで使用する Word ファイルを `../data/*` に格納しておきます。  
以下のコードでは、`../data/*` に格納された Word ファイルをすべて読み込み、テキストへ変換します。

In [ ]:
import glob
from langchain_community.document_loaders import Docx2txtLoader

files = glob.glob("../data/*.docx")
documents = []
for file in files:
    logger.debug(f"loaded file name: {file}")
    loader = Docx2txtLoader(file)
    document = loader.load()
    logger.debug(f"content: {document}")
    documents.extend(document)
logger.info(f"documents: {documents}")

以降では、読み込んだテキストデータをチャンクと呼ばれる単位に分割し、それをベクトル化したのちに Oracle Database 23ai へ格納していきます

In [ ]:
import oracledb
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings.oci_generative_ai import OCIGenAIEmbeddings
from langchain_community.document_loaders.oracleai import OracleTextSplitter

Oracle Database との接続に必要となるパラメータを `../.env` から読み込みます。  
以下のコードは、Autonomous Database が接続先の前提となっていますので、Base Database などに接続する場合は、以下のドキュメントを参考に、パラメータを一部修正してください。

参考: 

- [python-oracledb - 4. Connecting to Oracle Database](https://python-oracledb.readthedocs.io/en/latest/user_guide/connection_handling.html)

In [ ]:
# for Oracle Database 23ai
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
dsn = os.getenv("DSN")
config_dir = os.getenv("CONFIG_DIR")
wallet_dir = os.getenv("WALLET_DIR")
wallet_password = os.getenv("WALLET_PASSWORD")
table_name = os.getenv("TABLE_NAME")

logger.debug(f"username: {username}")
logger.debug(f"password: {password}")
logger.debug(f"dsn: {dsn}")
logger.debug(f"wallet dir: {wallet_dir}")
logger.debug(f"wallet password: {wallet_password}")
logger.debug(f"table name: {table_name}")

# for OCI Generative AI Service
compartment_id = os.getenv("COMPARTMENT_ID")
service_endpoint = os.getenv("SERVICE_ENDPOINT")

logger.debug(f"compartment id: {compartment_id}")
logger.debug(f"service endpoint: {service_endpoint}")

Oracle Database とのコネクションを作成します。

In [ ]:
connection = oracledb.connect(
    dsn=dsn,
    user=username,
    password=password,
    config_dir=config_dir,
    wallet_location=wallet_dir,
    wallet_password=wallet_password
)

データベースに格納する際のベクトルを得るための埋め込み関数とベクトルデータベースを作成します。

In [ ]:
embedding_function = OCIGenAIEmbeddings(
    auth_type="INSTANCE_PRINCIPAL",
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
)

oracle_vs = OracleVS(
    client=connection,
    embedding_function=embedding_function,
    table_name=table_name,
    distance_strategy=DistanceStrategy.COSINE,
    query="What is Oracle Database?"
)

cohere.embed-multilingual-v3.0 では、モデルの制約としてベクトル化対象の文章が最大 512 文字である必要があるため、読み込んだテキストをこれに収まるように分割（チャンキング）します。

In [ ]:
splitter_params = {"split": "recursively", "max": 300, "by": "words", "overlap": 30, "normalize": "all"}
splitter = OracleTextSplitter(conn=connection, params=splitter_params)

data = splitter.split_documents(documents=documents)

データベースにデータを追加します。

In [ ]:
oracle_vs.add_documents(documents=data)

簡易的な RAG のフローに沿って目的の回答が生成されることを確認します。

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

chat = ChatOCIGenAI(
    auth_type="INSTANCE_PRINCIPAL",
    service_endpoint=service_endpoint,
    compartment_id=compartment_id,
    model_id="cohere.command-r-plus",
    is_stream=True,
    model_kwargs={
        "temperature": 0,
        "max_tokens": 500,
        "top_p": 0.75,
        "top_k": 0,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

template = """
可能な限り、検索によって得られたコンテキストに則って回答を作成してください。
コンテキスト: {context}
---
質問: {query}
"""

prompt_template = PromptTemplate.from_template(
    template=template,
)

chain = (
    {"context": oracle_vs.as_retriever(), "query": RunnablePassthrough()}
    | prompt_template
    | chat
    | StrOutputParser()
)

res = chain.stream("海外出張って日当とかでるんでしたっけ？")

for chunk in res:
    print(chunk, end="")